In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# clean data file to only include relevant dates

csv_path = 'avo_change_dates.csv'
df = pd.read_csv(csv_path)

df_clean = df[df['Price Change']]
dates = df_clean['Date'].tolist()

In [3]:
def avoscraper(dates):
    avo_info = {}
    
    for date in dates:
        date_dt = datetime.strptime(date, '%Y-%m-%d')
        pre3m = date_dt - relativedelta(months = 3)
        post3m = date_dt + relativedelta(months = 3)
        
        text = 'avocado price changes before:{} after:{}'.format(post3m.strftime('%Y-%m-%d'), pre3m.strftime('%Y-%m-%d'))
        url = 'https://google.com/search?q=' + text
        
        request_result = requests.get(url)
        soup = BeautifulSoup(request_result.text, "html.parser")
        heading = soup.find_all('h3')   
        titles = [info.getText() for info in heading]
        avo_info[date] = titles
    
    return avo_info

avo_info = avoscraper(dates)
print(avo_info)

{'2015-01-04': ['An Avocado Success Story | Mercatus Center', '[PDF] THE U.S. MARKET FOR AVOCADO - USAID', 'Column: An avocado success story - Tampa Bay Times', '[PDF] Examining Consumer Preferences Within the Avocado Industry ...', "The rise of the avocado, America's new favorite fruit", '[PDF] Sample Avocado Production Costs and Profitability Analysis for ...', 'The Selling of the Avocado - The Atlantic', 'Avocados more expensive in Chile than in the Netherlands', "California's Drought Pushes Food Prices Higher", "You're Eating Too Many Avocados - National Geographic"], '2015-01-25': ['An Avocado Success Story | Mercatus Center', '[PDF] THE U.S. MARKET FOR AVOCADO - USAID', 'Column: An avocado success story - Tampa Bay Times', '[PDF] Examining Consumer Preferences Within the Avocado Industry ...', "The rise of the avocado, America's new favorite fruit", '[PDF] Sample Avocado Production Costs and Profitability Analysis for ...', 'The Selling of the Avocado - The Atlantic', 'Have You E

In [4]:
# turn dictionary into dataframe

df = pd.DataFrame({'Date': [date for date in avo_info for _ in avo_info[date]],
                   'Title': [title for titles in avo_info.values() for title in titles]})
avoscrape_df = df.pivot_table(index='Date', columns=df.groupby('Date').cumcount(), values='Title', aggfunc='first')
avoscrape_df = avoscrape_df.fillna('')
avoscrape_df

,0,1,2,3,4,5,6,7,8,9
Date,,,,,,,,,,
2015-01-04,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher,You're Eating Too Many Avocados - National Geo...
2015-01-25,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher
2015-02-01,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher,Have You Eaten Your Last Avocado?
2015-02-15,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,"Thanks to America, We've Reached Peak Avocado",The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,Avocados more expensive in Chile than in the N...
2015-03-01,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,"Thanks to America, We've Reached Peak Avocado",The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,
...,...,...,...,...,...,...,...,...,...,...
2018-01-21,How Price Affects Avocado Eaters - Econlife,China's middle class is boosting demand for av...,How Lower Avocado Prices Relate to Supply and ...,Avocado off the menu as shortage sends prices ...,Solved Consider the supply of avocados. Explai...,Mexico Fresh Avocado wholesale market price - ...,Avocado prices could spike thanks to a short s...,"Avocado prices to reach $4 as demand surges, s...",Avocado prices set to fall as fruit numbers sm...,
2018-01-28,How Price Affects Avocado Eaters - Econlife,China's middle class is boosting demand for av...,How Lower Avocado Prices Relate to Supply and ...,Avocado off the menu as shortage sends prices ...,Solved Consider the supply of avocados. Explai...,Mexico Fresh Avocado wholesale market price - ...,Avocado prices at record high due to reduced h...,Avocado prices could spike thanks to a short s...,"Avocado prices to reach $4 as demand surges, s...",
2018-02-04,How Price Affects Avocado Eaters - Econlife,China's middle class is boosting demand for av...,How Lower Avocado Prices Relate to Supply and ...,Avocado off the menu as shortage sends prices ...,Solved Consider the supply of avocados. Explai...,Mexico Fresh Avocado wholesale market price - ...,Avocado prices at record high due to reduced h...,"Avocado prices to reach $4 as demand surges, s...",Mexican avocado producers discredit NAFTA 'Ame...,


In [5]:
# combine new dataframe with combined_alldates.csv df

df = pd.read_csv('./combined_alldates.csv')
avo_combo = pd.merge(avoscrape_df, df, on = 'Date', how = 'outer')
# avo_combo = avo_combo.drop('dates', axis = 1)
avo_combo = avo_combo.fillna('')
display(avo_combo)

,Date,0,1,2,3,4,5,6,7,8,...,Unemployment Rate,Labor Force Participation Rate,Personal Consumption Expenditures,Trade Balance,Real Export,Personal Income,Personal Saving,quarter,year,date
0,2015-01-04,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-01-01
1,2015-01-25,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,Avocados more expensive in Chile than in the N...,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-01-01
2,2015-02-01,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-02-01
3,2015-02-15,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,"Thanks to America, We've Reached Peak Avocado",The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-02-01
4,2015-03-01,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,"Thanks to America, We've Reached Peak Avocado",The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2018-02-11,,,,,,,,,,...,3.9,62.9,13850.8,-44120.0,2.2,15493.0,1124.3,Q1,2018,2018-02-01
165,2018-02-25,,,,,,,,,,...,3.9,62.9,13850.8,-44120.0,2.2,15493.0,1124.3,Q1,2018,2018-02-01
166,2018-03-04,,,,,,,,,,...,3.9,62.9,13850.8,-44120.0,2.2,15493.0,1124.3,Q1,2018,2018-03-01
167,2018-03-11,,,,,,,,,,...,3.9,62.9,13850.8,-44120.0,2.2,15493.0,1124.3,Q1,2018,2018-03-01


In [6]:
avo_combo.to_csv('avo_combo_all.csv')

In [8]:
df = pd.read_csv('./combined_alldates.csv')
avo_combo = pd.merge(avoscrape_df, df, on = 'Date', how = 'inner')
# avo_combo = avo_combo.drop('dates', axis = 1)
avo_combo = avo_combo.fillna('')
display(avo_combo)

,Date,0,1,2,3,4,5,6,7,8,...,Unemployment Rate,Labor Force Participation Rate,Personal Consumption Expenditures,Trade Balance,Real Export,Personal Income,Personal Saving,quarter,year,date
0,2015-01-04,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-01-01
1,2015-01-25,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,Avocados more expensive in Chile than in the N...,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-01-01
2,2015-02-01,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,The Selling of the Avocado - The Atlantic,Avocados more expensive in Chile than in the N...,California's Drought Pushes Food Prices Higher,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-02-01
3,2015-02-15,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,"Thanks to America, We've Reached Peak Avocado",The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-02-01
4,2015-03-01,An Avocado Success Story | Mercatus Center,[PDF] THE U.S. MARKET FOR AVOCADO - USAID,Column: An avocado success story - Tampa Bay T...,[PDF] Examining Consumer Preferences Within th...,"The rise of the avocado, America's new favorit...",[PDF] Sample Avocado Production Costs and Prof...,"Thanks to America, We've Reached Peak Avocado",The Selling of the Avocado - The Atlantic,Have You Eaten Your Last Avocado?,...,5.4,62.8,12224.7,-40254.0,1.1,13696.9,1027.7,Q1,2015,2015-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2018-01-21,How Price Affects Avocado Eaters - Econlife,China's middle class is boosting demand for av...,How Lower Avocado Prices Relate to Supply and ...,Avocado off the menu as shortage sends prices ...,Solved Consider the supply of avocados. Explai...,Mexico Fresh Avocado wholesale market price - ...,Avocado prices could spike thanks to a short s...,"Avocado prices to reach $4 as demand surges, s...",Avocado prices set to fall as fruit numbers sm...,...,3.9,62.9,13850.8,-44120.0,2.2,15493.0,1124.3,Q1,2018,2018-01-01
85,2018-01-28,How Price Affects Avocado Eaters - Econlife,China's middle class is boosting demand for av...,How Lower Avocado Prices Relate to Supply and ...,Avocado off the menu as shortage sends prices ...,Solved Consider the supply of avocados. Explai...,Mexico Fresh Avocado wholesale market price - ...,Avocado prices at record high due to reduced h...,Avocado prices could spike thanks to a short s...,"Avocado prices to reach $4 as demand surges, s...",...,3.9,62.9,13850.8,-44120.0,2.2,15493.0,1124.3,Q1,2018,2018-01-01
86,2018-02-04,How Price Affects Avocado Eaters - Econlife,China's middle class is boosting demand for av...,How Lower Avocado Prices Relate to Supply and ...,Avocado off the men

In [9]:
avo_combo.to_csv('avo_combo_changes.csv')